<a href="https://colab.research.google.com/github/LauroML/Extrator_pLDDT_AF2_AF3/blob/main/extrator_de_plddt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.5 MB/s eta 0:00:00


Extrator de pLDDT ALPHAFOLD2



In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import re

def parse_pdb(file_path):
    """Parseia o arquivo PDB e extrai resíduos, números e valores de pLDDT."""
    residues = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('ATOM'):
                res_name = line[17:20].strip()
                res_num = int(line[22:26].strip())
                b_factor = float(line[60:66].strip())  # pLDDT está no campo B-factor
                residues.append({'res_name': res_name, 'res_num': res_num, 'pLDDT': b_factor})
    return pd.DataFrame(residues).drop_duplicates(subset=['res_num'])

def validate_residue_input(res_input):
    """Valida o formato do resíduo (ex.: ALA123)."""
    pattern = re.compile(r'^[A-Z]{3}\d+$')
    if not pattern.match(res_input):
        raise ValueError(f"Formato inválido: {res_input}. Use o formato 'ALA123' (nome do resíduo em 3 letras + número).")
    res_name = res_input[:3]
    res_num = int(res_input[3:])
    return res_name, res_num

def get_neighboring_residues(df, res_num, n=5):
    """Obtém os 5 resíduos à esquerda e à direita do resíduo especificado."""
    left_neighbors = df[df['res_num'].between(res_num - n, res_num - 1)]
    right_neighbors = df[df['res_num'].between(res_num + 1, res_num + n)]
    return pd.concat([left_neighbors, right_neighbors])

def main():
    # Upload do arquivo PDB
    print("Faça o upload do arquivo PDB da proteína gerado pelo AlphaFold2.")
    uploaded = files.upload()
    pdb_file = list(uploaded.keys())[0]

    # Parseia o arquivo PDB
    df = parse_pdb(pdb_file)

    # Solicita os 6 resíduos
    print("\nInsira 6 resíduos no formato 'ALA123' (nome do resíduo em 3 letras + número).")
    residues_input = []
    for i in range(6):
        while True:
            res = input(f"Resíduo {i+1}: ").strip().upper()
            try:
                res_name, res_num = validate_residue_input(res)
                if res_num not in df['res_num'].values:
                    print(f"Resíduo {res} não encontrado no arquivo PDB.")
                    continue
                residues_input.append((res_name, res_num))
                break
            except ValueError as e:
                print(e)

    # Dicionário para armazenar os resultados
    results_dict = {}

    # Lista para armazenar os dados do CSV
    csv_data = []

    # Processa cada resíduo inserido
    for res_name, res_num in residues_input:
        # Extrai o resíduo selecionado
        selected_res = df[df['res_num'] == res_num]
        if not selected_res.empty:
            plddt = selected_res['pLDDT'].iloc[0]
            results_dict[f"{res_name}{res_num}"] = plddt
            csv_data.append({
                'Residue': f"{res_name}{res_num}",
                'pLDDT': plddt,
                'Type': 'Selected'
            })

        # Extrai os 5 resíduos vizinhos à esquerda e à direita
        neighbors = get_neighboring_residues(df, res_num)
        for _, row in neighbors.iterrows():
            neighbor_key = f"{row['res_name']}{row['res_num']}"
            results_dict[neighbor_key] = row['pLDDT']
            csv_data.append({
                'Residue': neighbor_key,
                'pLDDT': row['pLDDT'],
                'Type': 'Neighbor'
            })

    # Cria o DataFrame para o CSV
    csv_df = pd.DataFrame(csv_data)
    csv_df.to_csv('plddt_output.csv', index=False)
    print("\nArquivo 'plddt_output.csv' gerado com sucesso!")
    files.download('plddt_output.csv')

    # Exibe o dicionário de resultados
    print("\nDicionário com os valores de pLDDT:")
    print(results_dict)

if __name__ == "__main__":
    main()

Faça o upload do arquivo PDB da proteína gerado pelo AlphaFold2.


Saving Maestro_hp2x7p_alphafold_trimerico.pdb to Maestro_hp2x7p_alphafold_trimerico.pdb

Insira 6 resíduos no formato 'ALA123' (nome do resíduo em 3 letras + número).
Resíduo 1: PHE293
Resíduo 2: PHE293
Resíduo 3: PHE293
Resíduo 4: PHE293
Resíduo 5: PHE293
Resíduo 6: PHE293

Arquivo 'plddt_output.csv' gerado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Dicionário com os valores de pLDDT:
{'PHE293': np.float64(88.56), 'TYR288': 56.69, 'PRO289': 64.56, 'GLY290': 78.75, 'TYR291': 87.94, 'ASN292': 90.5, 'ARG294': 91.25, 'TYR295': 89.44, 'ALA296': 92.56, 'LYS297': 91.44, 'TYR298': 91.25}


EXTRATOR de pLDDT ALPHAFOLD3

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import sys

try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def parse_pdb(file_path):
    """Parseia o arquivo PDB e extrai resíduos, números, cadeias e valores de pLDDT por átomo."""
    residues = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('ATOM'):
                    try:
                        chain_id = line[21].strip()  # Coluna 22 (índice 21 em Python)
                        res_name = line[17:20].strip()  # Colunas 18-20
                        res_num = int(line[22:26].strip())  # Colunas 23-26
                        b_factor = float(line[60:66].strip())  # Colunas 61-66 (pLDDT)
                        # Valida pLDDT (deve estar entre 0 e 100)
                        if not 0 <= b_factor <= 100:
                            print(f"Aviso: pLDDT inválido ({b_factor}) no resíduo {res_name}{res_num} da cadeia {chain_id}.")
                            continue
                        residues.append({
                            'chain_id': chain_id,
                            'res_name': res_name,
                            'res_num': res_num,
                            'pLDDT': b_factor
                        })
                    except (ValueError, IndexError):
                        print(f"Aviso: Linha malformada no PDB: {line.strip()}")
                        continue
        if not residues:
            raise ValueError("Nenhum registro ATOM válido encontrado no arquivo PDB.")
        return pd.DataFrame(residues)
    except FileNotFoundError:
        raise FileNotFoundError(f"Arquivo PDB não encontrado: {file_path}")

def validate_residue_input(res_input):
    """Valida o formato do resíduo (ex.: A:ALA123)."""
    pattern = re.compile(r'^[A-Z]:[A-Z]{3}\d+$')
    if not pattern.match(res_input):
        raise ValueError(f"Formato inválido: {res_input}. Use 'A:ALA123' (cadeia:nome do resíduo+número).")
    chain_id = res_input[0]
    res_name = res_input[2:5]
    res_num = int(res_input[5:])
    return chain_id, res_name, res_num

def get_neighboring_residues(df, chain_id, res_num, n=5):
    """Obtém os n resíduos à esquerda e à direita do resíduo especificado na mesma cadeia."""
    chain_df = df[df['chain_id'] == chain_id]
    left_neighbors = chain_df[chain_df['res_num'].between(res_num - n, res_num - 1)]
    right_neighbors = chain_df[chain_df['res_num'].between(res_num + 1, res_num + n)]
    return pd.concat([left_neighbors, right_neighbors])

def main():
    # Obtém o arquivo PDB
    if IN_COLAB:
        print("Faça o upload do arquivo PDB da proteína gerado pelo AlphaFold3.")
        uploaded = files.upload()
        pdb_file = list(uploaded.keys())[0]
    else:
        if len(sys.argv) != 2:
            print("Uso: python extract_plddt.py <caminho_do_arquivo_pdb>")
            sys.exit(1)
        pdb_file = sys.argv[1]

    # Parseia o arquivo PDB
    try:
        df_atoms = parse_pdb(pdb_file)
    except Exception as e:
        print(f"Erro ao parsear o arquivo PDB: {e}")
        sys.exit(1)

    # Agrega por resíduo e cadeia, calculando a média de pLDDT e mantendo os valores brutos
    df = (
        df_atoms
        .groupby(['chain_id', 'res_name', 'res_num'], as_index=False)
        .agg(
            pLDDT_mean=('pLDDT', 'mean'),
            pLDDT_raw=('pLDDT', list)  # Armazena os valores brutos de pLDDT como lista
        )
    )
    df['pLDDT_mean'] = df['pLDDT_mean'].round(2)  # Arredonda para 2 casas decimais

    # Solicita os 6 resíduos
    print("\nInsira 6 resíduos no formato 'A:ALA123' (cadeia:nome do resíduo+número).")
    residues_input = []
    for i in range(6):
        while True:
            res = input(f"Resíduo {i+1}: ").strip().upper()
            try:
                chain_id, res_name, res_num = validate_residue_input(res)
                if not ((df['chain_id'] == chain_id) & (df['res_num'] == res_num)).any():
                    print(f"Resíduo {res} não encontrado no arquivo PDB.")
                    continue
                residues_input.append((chain_id, res_name, res_num))
                break
            except ValueError as e:
                print(e)

    # Dicionário para armazenar os resultados
    results_dict = {}

    # Lista para armazenar os dados do CSV
    csv_data = []

    # Processa cada resíduo inserido
    for chain_id, res_name, res_num in residues_input:
        # Extrai o resíduo selecionado
        selected_res = df[(df['chain_id'] == chain_id) & (df['res_num'] == res_num)]
        if not selected_res.empty:
            plddt_mean = selected_res['pLDDT_mean'].iloc[0]
            plddt_raw = selected_res['pLDDT_raw'].iloc[0]
            res_key = f"{chain_id}:{res_name}{res_num}"
            results_dict[res_key] = plddt_mean
            csv_data.append({
                'Residue': res_key,
                'pLDDT_mean': plddt_mean,
                'pLDDT_raw': plddt_raw,
                'Type': 'Selected'
            })

        # Extrai os 5 resíduos vizinhos à esquerda e à direita
        neighbors = get_neighboring_residues(df, chain_id, res_num)
        for _, row in neighbors.iterrows():
            neighbor_key = f"{row['chain_id']}:{row['res_name']}{row['res_num']}"
            results_dict[neighbor_key] = row['pLDDT_mean']
            csv_data.append({
                'Residue': neighbor_key,
                'pLDDT_mean': row['pLDDT_mean'],
                'pLDDT_raw': row['pLDDT_raw'],
                'Type': 'Neighbor'
            })

    # Cria o DataFrame para o CSV
    csv_df = pd.DataFrame(csv_data)
    output_csv = 'plddt_output.csv'
    csv_df.to_csv(output_csv, index=False)
    print(f"\nArquivo '{output_csv}' gerado com sucesso!")

    if IN_COLAB:
        files.download(output_csv)

    # Exibe o dicionário de resultados
    print("\nDicionário com os valores de pLDDT:")
    print(results_dict)

if __name__ == "__main__":
    main()

Faça o upload do arquivo PDB da proteína gerado pelo AlphaFold3.


Saving melhor_modelo_alphafold3.pdb to melhor_modelo_alphafold3 (3).pdb

Insira 6 resíduos no formato 'A:ALA123' (cadeia:nome do resíduo+número).
Resíduo 1: A:LYS64
Resíduo 2: A:LYS66
Resíduo 3: A:THR189
Resíduo 4: A:ASN292
Resíduo 5: A:ARG294
Resíduo 6: A:LYS311

Arquivo 'plddt_output.csv' gerado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Dicionário com os valores de pLDDT:
{'A:LYS64': 82.49, 'A:HIS62': 84.38, 'A:SER59': 89.66, 'A:SER60': 88.71, 'A:THR63': 86.72, 'A:VAL61': 89.17, 'A:ALA69': 88.38, 'A:GLY67': 88.26, 'A:ILE68': 82.21, 'A:LYS66': np.float64(79.66), 'A:VAL65': 87.83, 'A:GLU70': 79.0, 'A:VAL71': 84.9, 'A:THR189': np.float64(87.48), 'A:ALA185': 90.07, 'A:ASN187': 85.81, 'A:GLU186': 85.38, 'A:PHE188': 90.25, 'A:SER184': 85.94, 'A:ASN194': 89.62, 'A:ILE192': 90.41, 'A:LEU191': 88.32, 'A:LYS193': 85.64, 'A:VAL190': 89.76, 'A:ASN292': 86.08, 'A:GLY290': 86.58, 'A:LEU287': 75.83, 'A:PRO289': 83.94, 'A:TYR288': 78.04, 'A:TYR291': 87.59, 'A:ALA296': 87.83, 'A:ARG294': np.float64(82.88), 'A:LYS297': 80.27, 'A:PHE293': 85.9, 'A:TYR295': 82.67, 'A:TYR298': 85.02, 'A:TYR299': 82.3, 'A:LYS311': np.float64(87.58), 'A:ARG307': 85.74, 'A:ILE310': 86.59, 'A:LEU309': 88.45, 'A:LYS306': 83.06, 'A:THR308': 85.48, 'A:ARG316': 87.16, 'A:GLY314': 91.59, 'A:ILE315': 90.22, 'A:PHE313': 89.72, 'A:VAL312': 88.89}
